<a href="https://colab.research.google.com/github/mmsamiei/thesis-prototype/blob/master/data_loading_phase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!cp /content/drive/My\ Drive/Thesis/phase-2/tokenized_test.json /content/train.json

In [0]:
from torch.utils.data import Dataset, DataLoader
import os
import torch
import json
from torch.utils.data.sampler import SubsetRandomSampler

In [0]:
class MyDataset(Dataset):
    """My dataset."""

    def __init__(self, json_file):
        """
        Args:
            json_file (string): Path to the json file with annotations.
        """
        self.dialogues = json.load(open(json_file))
        self.root_dir = json_file

        self.dialogues = sorted(self.dialogues, key=lambda x: len(x['history']))

    def __len__(self):
        return len(self.dialogues)

    def __getitem__(self, idx):
      
        
        history_lst = self.dialogues[idx]['history']
        true_lst = self.dialogues[idx]['true_sentence']
        false_lst = self.dialogues[idx]['false_sentenc']


        if(len(history_lst)>35):
          history_lst = history_lst[-35:]

        if(len(true_lst)>35):
          true_lst = true_lst[:35]
        
        if(len(false_lst)>35):
          false_lst = false_lst[:35]
        

        history = torch.LongTensor(history_lst)
        true_sample = torch.LongTensor(true_lst)
        false_sample = torch.LongTensor(false_lst)

        sample = {'history': history, 'true_sample': true_sample, 'false_sample': false_sample}

        return sample

In [0]:
dataset = MyDataset('train.json')

In [0]:
def my_collate_fn(batch):

  len_batch = len(batch)

  
  max_len_history = max([len(data['history']) for data in batch])
  max_len_true_sample = max([len(data['true_sample']) for data in batch])
  max_len_false_sample = max([len(data['false_sample']) for data in batch])
  
  padding_ind = 1 ## for bert is 0 but because embedding hate 0!! we used 1
  result_history = torch.ones(len_batch, max_len_history)
  result_true_sample = torch.ones(len_batch, max_len_true_sample)
  result_false_sample = torch.ones(len_batch, max_len_false_sample)

  for i, data in enumerate(batch):
    p1 = len(data['history'])
    result_history[i, :p1] = data['history']
    p2 = len(data['true_sample'])
    result_true_sample[i, :p2] = data['true_sample']
    p3 = len(data['false_sample'])
    result_false_sample[i, :p3] = data['false_sample']



  return result_history.T.long(), result_true_sample.T.long(), result_false_sample.T.long()

sampler = torch.utils.data.SequentialSampler(dataset)
dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=32, sampler=sampler,
                                             shuffle=False, collate_fn=my_collate_fn)

In [0]:
for _, batch in enumerate(dataset_loader):
  history, true_sample, false_sample = batch

# Model

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [0]:
import numpy as np
import torch.nn as nn

class RepresentModel(nn.Module):
  
  def __init__(self, hid_size, vocab_size, n_head, n_layers, pf_size, max_len, device):
    super().__init__()

    self.device = device
    
    self.hid_size = hid_size
    self.pf_size = pf_size
    self.max_len = max_len
    self.n_head = n_head

    self.embedding = nn.Embedding(vocab_size, hid_size)

    self.position_enc = nn.Embedding(self.max_len, self.hid_size)
    self.position_enc.weight.data = self.position_encoding_init(self.max_len, self.hid_size)
    self.scale = torch.sqrt(torch.FloatTensor([self.hid_size])).to(device)

    self.layer_norm = nn.LayerNorm(self.hid_size)
    self.encoder_layer = nn.TransformerEncoderLayer(d_model=hid_size, nhead = n_head, dim_feedforward = pf_size)
    self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=n_layers, norm=self.layer_norm)

    self._init_weights()
  
  def forward(self, x):
    sent_len, batch_size = x.shape[0], x.shape[1]

    temp = x
    temp = self.embedding(temp)

    pos = torch.arange(1,sent_len+1).unsqueeze(1).repeat(1,batch_size).to(self.device)
    temp_pos_emb = self.position_enc(pos)

    temp = temp * self.scale + temp_pos_emb
    temp = self.encoder(temp)
    temp = temp[0,:]
    return temp

  def _init_weights(self):
    for p in self.parameters():
      if p.dim() > 1:
        nn.init.xavier_uniform_(p)

  def append_decoder_layer(self):
    appended_mod = nn.TransformerEncoderLayer(d_model=self.hid_size, nhead = self.n_head, dim_feedforward = self.pf_size).to(self.device)
    for p in appended_mod.parameters():
      if p.dim() > 1:
        nn.init.xavier_uniform_(p)
    self.encoder.layers.append(appended_mod)
    self.encoder.num_layers += 1

  
  def position_encoding_init(self, n_position, d_pos_vec):
    ''' Init the sinusoid position encoding table '''

    # keep dim 0 for padding token position encoding zero vector
    position_enc = np.array([
        [pos / np.power(10000, 2*i/d_pos_vec) for i in range(d_pos_vec)]
        if pos != 0 else np.zeros(d_pos_vec) for pos in range(n_position)])

    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2]) # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2]) # dim 2i+1
    temp = torch.from_numpy(position_enc).type(torch.FloatTensor)
    temp = temp.to(self.device)
    return temp

In [0]:
hid_size = 16
vocab_size = 34000 
n_head = 8
n_layers = 1
pf_size = 32
max_len = 500
model = RepresentModel(hid_size, vocab_size, n_head, n_layers, pf_size, max_len, device).to(device)

In [13]:
test_len = 20
batch_size = 64
test_input = torch.LongTensor(test_len, batch_size).random_(1,vocab_size).to(device)
model(test_input).shape

torch.Size([64, 16])

In [14]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 556,480 trainable parameters


In [0]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
    
    def zero_grad(self):
        self.optimizer.zero_grad()

In [16]:
!pip install -U tqdm

Requirement already up-to-date: tqdm in /usr/local/lib/python3.6/dist-packages (4.41.1)


In [0]:
from tqdm.notebook import tqdm

def train_one_epoch(model,train_iter, optimizer, criterion, clip):
  epoch_loss = 0
  model.train()
  for batch in tqdm(train_iter):
    optimizer.zero_grad()
    history, true_sample, false_sample = batch
    batch_size = history.shape[1]
    ### hist = [sent_len, batch]
    history_rpr = model(history.to(device))
    true_rpr = model(true_sample.to(device))
    false_rpr = model(false_sample.to(device))
    ## rpr = [batch_size, hidden]
    cos = nn.PairwiseDistance().to(device)
    tru_sml = -1*cos(history_rpr, true_rpr)
    fls_sml = -1*cos(history_rpr, false_rpr)
    mini_batch_tensor = torch.ones(batch_size).to(device)
    loss = criterion(tru_sml, fls_sml, mini_batch_tensor)
    print(history.shape[0], true_sample.shape[0], false_sample.shape[0])
    print(loss.item())
    loss.backward()
    epoch_loss += loss
  return epoch_loss / len(train_iter)

In [0]:
def train(model, train_iter, optimizer, criterion, clip, N_EPOCH):
  for epoch in range(N_EPOCH):
    epoch_loss = train_one_epoch(model, train_iter, optimizer, criterion, clip)


In [19]:
optimizer = NoamOpt(hid_size, 1, 2000,
              torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
criterion = torch.nn.MarginRankingLoss(margin=1).to(device)
train(model, dataset_loader, optimizer, criterion, 1, 1)

3 52 67
0.6460055708885193
3 52 56
1.054274559020996
5 52 62
0.7019585371017456
5 52 75
0.8209887742996216
6 34 87
0.9983526468276978
6 33 81
0.7971975803375244
6 43 80
0.8814077377319336
6 43 61
0.9948355555534363
6 35 57
1.0315334796905518
6 27 50
0.831860363483429
6 32 71
0.8117556571960449
7 37 85
0.7745225429534912
7 37 91
0.9712182283401489
7 39 65
0.9871038198471069
8 46 49
0.7969189882278442
8 52 80
0.7927400469779968
8 52 91
0.9067511558532715
8 95 68
0.9709776043891907
8 95 69
0.802191972732544
8 33 84
0.7575780153274536
8 48 86
0.876212477684021
8 48 81
0.6301506757736206
8 48 105
0.5902564525604248
8 40 60
0.9121924042701721
8 52 114
0.6895809173583984
8 52 99
0.7835640907287598
8 60 65
0.8980699777603149
8 60 71
0.9852886199951172
8 58 70
0.9208000302314758
8 58 68
0.9007561802864075
8 35 53
0.8275817632675171
8 27 46
0.9727283716201782
8 31 69
0.7444924116134644
8 48 77
0.704339861869812
8 48 51
0.6901280879974365
8 44 70
0.7601829767227173
8 44 60
1.0590320825576782
8 48

RuntimeError: ignored